# Freshwater eutrophication impact factors 

Exiobase categories:
- P - agriculture - water
- P - agriculture - soil

LC-Impact stressors
- Freshwater eutrophication

In [20]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc
import json
import os

# Load configuration from arguments.json
args_path = os.path.join(os.path.dirname(os.getcwd()), "arguments.json")
with open(args_path, 'r') as f:
    config = json.load(f)

# Extract paths from config
lc_impact_path = config['lc_impact_path']
exio_19_path = config['exio_19_path']
exio_11_path = config['exio_11_path']
row_region_mappings = config['row_region_mappings']
exiobase_grouping_patterns = config['exiobase_grouping_patterns']

In [21]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path=exio_11_path)
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path=exio_19_path)

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [ ]:
# no need for aggregation

In [17]:
# diagonalize stressor
# optimized for speed
diag_water = exio3_11.satellite.diag_stressor(("P - agriculture - water"))

# First, make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

# calculate S (direct emission multipliers)
if diag_water.S is None:
    print("Calculating S")
    diag_water.S = pymrio.calc_S(diag_water.F, exio3_11.x)

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

diag_water.D_cba, _, _, _ = pymrio.calc_accounts(diag_water.S, exio3_11.L, Y_agg)
diag_water.D_cba

L already loaded
Calculating S


/var/folders/bg/8zzl9cj57fnd569zy539k5h40000gn/T/ipykernel_51394/3100989159.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


region                                                              AT  \
sector                                                      Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              1.642010e-05   
       Cereal grains nec                                  7.913379e-05   
       Vegetables, fruit, nuts                            1.282658e-06   
       Oil seeds                                          4.414921e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       1.255401e-06   
       Recreational, cultural and sporting services (92)  5.748289e-05   
       Other services (93)                                5.653417e-06   
       Private households with employed persons (95)      2.322971e-07   
       Extra-territorial organizations and bodies         0.000000e+00   

region                                                                 \
sector                                                          Wheat   
region sector                                                           
AT     Paddy rice                                            0.000000   
       Wheat                                              1561.729034   
       Cereal grains nec                                     0.087185   
       Vegetables, fruit, nuts                               0.005508   
       Oil seeds                                             0.011053   
...                                                               ...   
WM     Membership organisation services n.e.c. (91)          0.000160   
       Recreational, cultural and sporting services (92)     0.008750   
       Other services (93)                                   0.000585   
       Private households with employed persons (95)         0.000036   
       Extra-territorial organizations and bodies            0.000000   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                                 0.000000   
       Wheat                                                      0.214414   
       Cereal grains nec                                      15138.007106   
       Vegetables, fruit, nuts                                    0.032290   
       Oil seeds                                                  0.054767   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)               0.000530   
       Recreational, cultural and sporting services (92)          0.032921   
       Other services (93)                                        0.001988   
       Private households with employed persons (95)              0.000129   
       Extra-territorial organizations and bodies                 0.000000   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                       0.000000   
       Wheat                                                            1.857819   
       Cereal grains nec                                                5.009631   
       Vegetables, fruit, nuts                                      14517.996488   
       Oil seeds                                                        0.382852   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                     0.023843   
       

In [8]:
# diagonalize stressor
# optimized for speed
diag_soil = exio3_11.satellite.diag_stressor(("P - agriculture - soil"))

# First, make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

# calculate S (direct emission multipliers)
if diag_soil.S is None:
    print("Calculating S")
    diag_soil.S = pymrio.calc_S(diag_soil.F, exio3_11.x)

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

diag_soil.D_cba, _, _, _ = pymrio.calc_accounts(diag_soil.S, exio3_11.L, Y_agg)
diag_soil.D_cba

L already loaded
Calculating S


/var/folders/bg/8zzl9cj57fnd569zy539k5h40000gn/T/ipykernel_51394/940356300.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


region                                                           AT  \
sector                                                   Paddy rice   
region sector                                                         
AT     Paddy rice                                          0.000000   
       Wheat                                               0.000550   
       Cereal grains nec                                   0.002650   
       Vegetables, fruit, nuts                             0.000043   
       Oil seeds                                           0.000148   
...                                                             ...   
WM     Membership organisation services n.e.c. (91)        0.000042   
       Recreational, cultural and sporting services (92)   0.001925   
       Other services (93)                                 0.000189   
       Private households with employed persons (95)       0.000008   
       Extra-territorial organizations and bodies          0.000000   

region                                                                  \
sector                                                           Wheat   
region sector                                                            
AT     Paddy rice                                             0.000000   
       Wheat                                              52290.996265   
       Cereal grains nec                                      2.919179   
       Vegetables, fruit, nuts                                0.184417   
       Oil seeds                                              0.370069   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)           0.005368   
       Recreational, cultural and sporting services (92)      0.292978   
       Other services (93)                                    0.019599   
       Private households with employed persons (95)          0.001221   
       Extra-territorial organizations and bodies             0.000000   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                                 0.000000   
       Wheat                                                      7.179156   
       Cereal grains nec                                     506862.237932   
       Vegetables, fruit, nuts                                    1.081154   
       Oil seeds                                                  1.833754   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)               0.017736   
       Recreational, cultural and sporting services (92)          1.102274   
       Other services (93)                                        0.066558   
       Private households with employed persons (95)              0.004329   
       Extra-territorial organizations and bodies                 0.000000   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                       0.000000   
       Wheat                                                           62.204908   
       Cereal grains nec                                              167.736273   
       Vegetables, fruit, nuts                                     486102.572053   
       Oil seeds                                                       12.818952   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                     0.798328   
       Recreational, cultural and s

## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [18]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_water.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s_w = pd.DataFrame(columns)
dr_s_w

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              9.732484e-08   
       Cereal grains nec                                  4.690399e-07   
       Vegetables, fruit, nuts                            7.602540e-09   
       Oil seeds                                          2.616801e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       7.440984e-09   
       Recreational, cultural and sporting services (92)  3.407112e-07   
       Other services (93)                                3.350879e-08   
       Private households with employed persons (95)      1.376866e-09   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              6.371025e-01   
       Cereal grains nec                                  3.556667e-05   
       Vegetables, fruit, nuts                            2.246901e-06   
       Oil seeds                                          4.508837e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       6.540363e-08   
       Recreational, cultural and sporting services (92)  3.569580e-06   
       Other services (93)                                2.387909e-07   
       Private households with employed persons (95)      1.487512e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.035024e-05   
       Cereal grains nec                                      7.307470e-01   
       Vegetables, fruit, nuts                                1.558708e-06   
       Oil seeds                                              2.643736e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           2.556949e-08   
       Recreational, cultural and sporting services (92)      1.589157e-06   
       Other services (93)                                    9.595777e-08   
       Private households with employed persons (95)          6.241534e-09   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        4.254184e-05   
       Cereal grains nec                                            1.147146e-04   
       Vegetables, fruit, nuts                                      3.324448e-01   
       Oil seeds                                                    8.766861e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 5.459750e-07   
       Recreational, cultural and sporting services (92)     

In [19]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_soil.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s_s = pd.DataFrame(columns)
dr_s_s

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              9.732484e-08   
       Cereal grains nec                                  4.690399e-07   
       Vegetables, fruit, nuts                            7.602540e-09   
       Oil seeds                                          2.616801e-08   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       7.440984e-09   
       Recreational, cultural and sporting services (92)  3.407112e-07   
       Other services (93)                                3.350879e-08   
       Private households with employed persons (95)      1.376866e-09   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              6.371025e-01   
       Cereal grains nec                                  3.556667e-05   
       Vegetables, fruit, nuts                            2.246901e-06   
       Oil seeds                                          4.508837e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       6.540363e-08   
       Recreational, cultural and sporting services (92)  3.569580e-06   
       Other services (93)                                2.387909e-07   
       Private households with employed persons (95)      1.487512e-08   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.035024e-05   
       Cereal grains nec                                      7.307470e-01   
       Vegetables, fruit, nuts                                1.558708e-06   
       Oil seeds                                              2.643736e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           2.556949e-08   
       Recreational, cultural and sporting services (92)      1.589157e-06   
       Other services (93)                                    9.595777e-08   
       Private households with employed persons (95)          6.241534e-09   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        4.254184e-05   
       Cereal grains nec                                            1.147146e-04   
       Vegetables, fruit, nuts                                      3.324448e-01   
       Oil seeds                                                    8.766861e-06   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 5.459750e-07   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [11]:
# Use row region mappings from config instead of hardcoded values
row_eu_countries = row_region_mappings["row_eu"]
row_asia_pacific_countries = row_region_mappings["row_asia_pacific"]
row_african_countries = row_region_mappings["row_africa"]
row_american_countries = row_region_mappings["row_america"]
row_middle_eastern_countries = row_region_mappings["row_middle_east"]

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# harmonize regions between lc-impact and exiobase
all_row_region_keys = list(row_eu_countries.keys()) + list(row_asia_pacific_countries.keys()) + list(row_african_countries.keys()) + list(row_american_countries.keys()) + list(row_middle_eastern_countries.keys())
exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
print(len(exio_regions_without_row)) # exiobase contains 44 exact regions + 5 row regions

# combine exact regions from exiobase with exact row regions
all_regions = list(all_row_region_keys) + list(exio_regions_without_row)
print("Regions: ", len(all_regions))
print("Unique regions: ", len(set(all_regions)))
assert len(all_regions) == len(set(all_regions)), "There are duplicate regions in the all_regions list"

44
Regions:  248
Unique regions:  248


In [12]:
# load and prepare lc-impact data for freshwater eutrophication using path from config
lci = pd.read_excel(f"{lc_impact_path}/8-freshwater eutrophication/CF_FWEutrophication.xlsx", sheet_name="Country CFs")
# take average of phosphorus emissions to water and soil
lci["Average"] = lci[["CF for P emissions to water [PDFyr/kg]", "CF for P emissions to soil [PDFyr/kg]"]].mean(axis=1)
lci.drop(columns=["CF from erosion [PDFyr/m2yr]"], inplace=True)
lci

,Country,CF for P emissions to water [PDFyr/kg],CF for P emissions to soil [PDFyr/kg],Average
0,Afghanistan,7.656778e-12,1.063088e-12,4.359933e-12
1,Albania,5.200098e-13,7.508058e-14,2.975452e-13
2,Algeria,6.360048e-15,6.435278e-16,3.501788e-15
3,Andorra,3.370127e-13,3.111552e-14,1.840641e-13
4,Angola,4.727749e-12,9.038396e-13,2.815794e-12
...,...,...,...,...
162,Western Sahara,5.498873e-13,4.261535e-14,2.962513e-13
163,Yemen,3.315156e-15,2.709144e-16,1.793035e-15
164,Zaire,3.690181e-12,3.874239e-13,2.038803e-12
165,Zambia,4.077208e-12,4.462874e-13,2.261748e-12


In [13]:
# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Country code not found for ", name)
            return None  # Return None if country not found

extra_mappings = {
    "Turkey": "TR",
    "Russia": "RU",
    "Bahamas, The": "BS",
    "Brunei": "BN",
    "Byelarus": "BY",
    "Cape Verde": "CV",
    "Congo DRC": "CD",
    "Falkland Islands (Islas Malvinas)": "FK",
    "Gambia, The": "GM",
    "Ivory Coast": "CI",
    "Macedonia": "MK",
    "Myanmar (Burma)": "MM",
    "Reunion": "RE",
    "St. Vincent and the Grenadines": "VC",
    "Svalbard": "SJ",
    "Swaziland": "SZ",
    "Virgin Islands": "VG",
    "Western Samoa": "WS",
}

lci["Country_Code"] = lci["Country"].apply(get_country_code)
# remove rows with missing country codes
lci = lci.dropna(subset=["Country_Code"])
# malta is missing from lc-impact, add malta as new row with country code MT and EU averages
p_w_eu = 2.19740701963604E-13
p_s_eu = 2.27816881528299E-14
p_avg_eu = (p_w_eu + p_s_eu) / 2
malta_row = pd.DataFrame({
    "Country": ["Malta"],
    "CF for P emissions to water [PDFyr/kg]": [p_w_eu],
    "CF for P emissions to soil [PDFyr/kg]": [p_s_eu],
    "Average": [p_avg_eu],
    "Country_Code": ["MT"],
})
lci = pd.concat([lci, malta_row], ignore_index=True)
# cyprus is missing from lc-impact, add cyprus as new row with country code CY and EU averages
cyprus_row = pd.DataFrame({
    "Country": ["Cyprus"],
    "CF for P emissions to water [PDFyr/kg]": [p_w_eu],
    "CF for P emissions to soil [PDFyr/kg]": [p_s_eu],
    "Average": [p_avg_eu],
    "Country_Code": ["CY"],
})
lci = pd.concat([lci, cyprus_row], ignore_index=True)
lci.set_index("Country_Code", inplace=True)
lci

Country code not found for  Gaza Strip
Country code not found for  West Bank
Country code not found for  Zaire


,Country,CF for P emissions to water [PDFyr/kg],CF for P emissions to soil [PDFyr/kg],Average
Country_Code,,,,
AF,Afghanistan,7.656778e-12,1.063088e-12,4.359933e-12
AL,Albania,5.200098e-13,7.508058e-14,2.975452e-13
DZ,Algeria,6.360048e-15,6.435278e-16,3.501788e-15
AD,Andorra,3.370127e-13,3.111552e-14,1.840641e-13
AO,Angola,4.727749e-12,9.038396e-13,2.815794e-12
...,...,...,...,...
YE,Yemen,3.315156e-15,2.709144e-16,1.793035e-15
ZM,Zambia,4.077208e-12,4.462874e-13,2.261748e-12
ZW,Zimbabwe,1.401582e-11,1.223770e-12,7.619793e-12


In [14]:
# find if some regions are missing from lci
missing_regions = [region for region in all_regions if region not in lci.index.unique()]
print("Missing regions (lc-impact): ", missing_regions)

# find out if some regions are missing from augmented exiobase regions 
lci_regions = lci.index
missing_from_all_regions = [region for region in lci_regions if region not in all_regions]
print("Missing regions (exiobase, augmented): ", missing_from_all_regions)

# assert that all regions in lci-water are in the all_regions list
assert sorted(lci_regions) == sorted(all_regions), "Some regions in lci-water are not in the all_regions list"

Missing regions (lc-impact):  ['FO', 'GG', 'GI', 'GL', 'IM', 'JE', 'MC', 'SJ', 'AS', 'IO', 'CX', 'CC', 'CK', 'FJ', 'PF', 'TF', 'GU', 'HK', 'KI', 'MO', 'MV', 'MH', 'FM', 'NR', 'NC', 'NU', 'NF', 'MP', 'PW', 'PN', 'WS', 'SB', 'TL', 'TK', 'TO', 'TV', 'UM', 'VU', 'WF', 'CV', 'KM', 'CD', 'HM', 'MU', 'RE', 'ST', 'SH', 'SC', 'SS', 'YT', 'AQ', 'AI', 'AG', 'AN', 'AW', 'BS', 'BL', 'BB', 'BQ', 'BM', 'BV', 'CW', 'DM', 'GP', 'GS', 'GD', 'JM', 'FK', 'KY', 'MS', 'MF', 'PM', 'PR', 'KN', 'LC', 'SX', 'TC', 'VI', 'VG', 'VC', 'MQ', 'PS']
Missing regions (exiobase, augmented):  []


AssertionError: Some regions in lci-water are not in the all_regions list

In [54]:
# augment dr_s_w to create dr_u_w

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s_w.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s_w.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s_w.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s_w.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s_w.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u_w = dr_s_w.copy()
dr_u_w = dr_u_w.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in all_row_region_keys:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u_w
all_indices = []
all_data = []
for region in all_row_region_keys:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_w = pd.concat([dr_u_w, combined_data])

In [66]:
# augment dr_s_s to create dr_u_s

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s_s.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s_s.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s_s.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s_s.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s_s.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u_s = dr_s_s.copy()
dr_u_s = dr_u_s.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in all_row_region_keys:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u_s
all_indices = []
all_data = []
for region in all_row_region_keys:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_s = pd.concat([dr_u_s, combined_data])

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [77]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f_w = dr_u_w.copy()
p_agriculture_water = exio3_19.satellite.M.loc["P - agriculture - water"]
scalars_w = p_agriculture_water.to_numpy() # multipliers for each column
print(len(scalars_w))
print(dr_u_w.shape)

# multiply each column of dr_u_w by the respective column value from exio3_19.satellite_agg.M
dr_f_w = dr_f_w * scalars_w # same as dr_f_w * diag(scalars) but more efficient with numpy broadcasting
dr_f_w

9800
(33200, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  6.218397e+01   
   Cereal grains nec                                        0.0  3.471461e-03   
   Vegetables, fruit, nuts                                  0.0  2.193073e-04   
   Oil seeds                                                0.0  4.400820e-04   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  4.255786e-07   
   Recreational, cultural and sporting services (92)        0.0  2.322710e-05   
   Other services (93)                                      0.0  1.553802e-06   
   Private households with employed persons (95)            0.0  9.679173e-08   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.038944e-03   
   Cereal grains nec                                      7.335142e+01   
   Vegetables, fruit, nuts                                1.564610e-04   
   Oil seeds                                              2.653748e-04   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           1.711088e-07   
   Recreational, cultural and sporting services (92)      1.063450e-05   
   Other services (93)                                    6.421409e-07   
   Private households with employed persons (95)          4.176780e-08   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        8.147880e-04   
   Cereal grains nec                                            2.197086e-03   
   Vegetables, fruit, nuts                                      6.367191e+00   
   Oil seeds                                                    1.679084e-04   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 6.971237e-07   
   Recreational, cultural and sporting services (92)            1.010264e-05   
   Other services (93)                                          1.196105e-06   
   Private households with employed persons (95)                3.030816e-08   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              1.153229e-04   
   Cereal grains nec                                  2.208263e-04   
   Vegetables, fruit, nuts                            1.599124e-05   
   Oil seeds                                          5.198346e-01   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       1.765932e-07   
   Recreational, cultural and sporting services (92)  4.766529e-06   
   Other services (93)                                3.820907e-07   
   Private households with employed persons (95)      3.133883e-08   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [73]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f_s = dr_u_s.copy()
p_agriculture_soil = exio3_19.satellite.M.loc["P - agriculture - soil"]
scalars_soil = p_agriculture_soil.to_numpy() # multipliers for each column
print(len(scalars_soil))
print(dr_u_s.shape)

# multiply each column of dr_u_w by the respective column value from exio3_19.satellite_agg.M
dr_f_s = dr_f_s * scalars_soil # same as dr_f_s * diag(scalars) but more efficient with numpy broadcasting
dr_f_s

9800
(33200, 9800)


AT               \
                                                     Paddy rice        Wheat   
AT Paddy rice                                               0.0     0.000000   
   Wheat                                                    0.0  2082.090747   
   Cereal grains nec                                        0.0     0.116234   
   Vegetables, fruit, nuts                                  0.0     0.007343   
   Oil seeds                                                0.0     0.014735   
...                                                         ...          ...   
YE Membership organisation services n.e.c. (91)             0.0     0.000014   
   Recreational, cultural and sporting services (92)        0.0     0.000778   
   Other services (93)                                      0.0     0.000052   
   Private households with employed persons (95)            0.0     0.000003   
   Extra-territorial organizations and bodies               0.0     0.000000   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                                 0.000000   
   Wheat                                                      0.034787   
   Cereal grains nec                                       2456.008052   
   Vegetables, fruit, nuts                                    0.005239   
   Oil seeds                                                  0.008885   
...                                                                ...   
YE Membership organisation services n.e.c. (91)               0.000006   
   Recreational, cultural and sporting services (92)          0.000356   
   Other services (93)                                        0.000022   
   Private households with employed persons (95)              0.000001   
   Extra-territorial organizations and bodies                 0.000000   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                       0.000000   
   Wheat                                                            0.027281   
   Cereal grains nec                                                0.073564   
   Vegetables, fruit, nuts                                        213.191131   
   Oil seeds                                                        0.005622   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                     0.000023   
   Recreational, cultural and sporting services (92)                0.000338   
   Other services (93)                                              0.000040   
   Private households with employed persons (95)                    0.000001   
   Extra-territorial organizations and bodies                       0.000000   

                                                                 \
                                                      Oil seeds   
AT Paddy rice                                          0.000000   
   Wheat                                               0.003861   
   Cereal grains nec                                   0.007394   
   Vegetables, fruit, nuts                             0.000535   
   Oil seeds                                          17.405496   
...                                                         ...   
YE Membership organisation services n.e.c. (91)        0.000006   
   Recreational, cultural and sporting services (92)   0.000160   
   Other services (93)                                 0.000013   
   Private households with employed persons (95)       0.000001   
   Extra-territorial organizations and bodies          0.000000   

                                                                             \
                                                     Sugar cane, 

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [78]:
# sort rows on lci in same order as dr_f.index.sortlevel
lci = lci.reindex(dr_f_w.index.get_level_values(0).unique())

# using average of P emissions to water and soil
# is this correct?
cf_avg = lci["Average"].to_numpy()
cf_avg = np.repeat(cf_avg, 200)
# expand cf_all_effects to match the shape of dr_f
cf_avg = np.tile(cf_avg, (dr_f_w.shape[1], 1)).T
print(cf_avg.shape) # (37000, 9800)

# multiply each column of dr_f with cf_all_effects
pdf_w = dr_f_w * cf_avg
pdf_w

(33200, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  7.932818e-12   
   Cereal grains nec                                        0.0  4.428548e-16   
   Vegetables, fruit, nuts                                  0.0  2.797706e-17   
   Oil seeds                                                0.0  5.614133e-17   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  7.630774e-22   
   Recreational, cultural and sporting services (92)        0.0  4.164701e-20   
   Other services (93)                                      0.0  2.786022e-21   
   Private households with employed persons (95)            0.0  1.735510e-22   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.325382e-16   
   Cereal grains nec                                      9.357453e-12   
   Vegetables, fruit, nuts                                1.995976e-17   
   Oil seeds                                              3.385390e-17   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           3.068041e-22   
   Recreational, cultural and sporting services (92)      1.906803e-20   
   Other services (93)                                    1.151381e-21   
   Private households with employed persons (95)          7.489114e-23   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        1.039426e-16   
   Cereal grains nec                                            2.802825e-16   
   Vegetables, fruit, nuts                                      8.122636e-13   
   Oil seeds                                                    2.142010e-17   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 1.249968e-21   
   Recreational, cultural and sporting services (92)            1.811439e-20   
   Other services (93)                                          2.144658e-21   
   Private households with employed persons (95)                5.434361e-23   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              1.471176e-17   
   Cereal grains nec                                  2.817085e-17   
   Vegetables, fruit, nuts                            2.040004e-18   
   Oil seeds                                          6.631538e-14   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       3.166379e-22   
   Recreational, cultural and sporting services (92)  8.546556e-21   
   Other services (93)                                6.851021e-22   
   Private households with employed persons (95)      5.619163e-23   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [79]:
# sum up all columns to get the total impact for each region-sector pair
pdf_total_w = pdf_w.sum()
pdf_total_w

AT  Paddy rice                                           0.000000e+00
    Wheat                                                2.639687e-11
    Cereal grains nec                                    1.796384e-11
    Vegetables, fruit, nuts                              1.542217e-11
    Oil seeds                                            2.785921e-11
                                                             ...     
WM  Membership organisation services n.e.c. (91)         4.623666e-12
    Recreational, cultural and sporting services (92)    1.354785e-11
    Other services (93)                                  2.674241e-11
    Private households with employed persons (95)        4.740957e-12
    Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [80]:
pd.DataFrame(pdf_total_w).to_csv("csv/pdf-eutrophication-freshwater-water.csv", index=True)

In [74]:
# sort rows on lci in same order as dr_f.index.sortlevel
lci = lci.reindex(dr_f_s.index.get_level_values(0).unique())

# using average of P emissions to water and soil
cf_avg = lci["Average"].to_numpy()
cf_avg = np.repeat(cf_avg, 200)
# expand cf_all_effects to match the shape of dr_f
cf_avg = np.tile(cf_avg, (dr_f_s.shape[1], 1)).T
print(cf_avg.shape) # (37000, 9800)

# multiply each column of dr_f with cf_all_effects
pdf_s = dr_f_s * cf_avg
pdf_s

(33200, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  2.656126e-10   
   Cereal grains nec                                        0.0  1.482800e-14   
   Vegetables, fruit, nuts                                  0.0  9.367493e-16   
   Oil seeds                                                0.0  1.879767e-15   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  2.554994e-20   
   Recreational, cultural and sporting services (92)        0.0  1.394457e-18   
   Other services (93)                                      0.0  9.328371e-20   
   Private households with employed persons (95)            0.0  5.810966e-21   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  4.437745e-15   
   Cereal grains nec                                      3.133133e-10   
   Vegetables, fruit, nuts                                6.683078e-16   
   Oil seeds                                              1.133522e-15   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           1.027265e-20   
   Recreational, cultural and sporting services (92)      6.384502e-19   
   Other services (93)                                    3.855143e-20   
   Private households with employed persons (95)          2.507562e-21   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        3.480286e-15   
   Cereal grains nec                                            9.384633e-15   
   Vegetables, fruit, nuts                                      2.719683e-11   
   Oil seeds                                                    7.172042e-16   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 4.185236e-20   
   Recreational, cultural and sporting services (92)            6.065198e-19   
   Other services (93)                                          7.180906e-20   
   Private households with employed persons (95)                1.819574e-21   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              4.925904e-16   
   Cereal grains nec                                  9.432377e-16   
   Vegetables, fruit, nuts                            6.830497e-17   
   Oil seeds                                          2.220422e-12   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       1.060191e-20   
   Recreational, cultural and sporting services (92)  2.861623e-19   
   Other services (93)                                2.293911e-20   
   Private households with employed persons (95)      1.881451e-21   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [75]:
# sum up all columns to get the total impact for each region-sector pair
pdf_total_s = pdf_s.sum()
pdf_total_s

AT  Paddy rice                                           0.000000e+00
    Wheat                                                8.838402e-10
    Cereal grains nec                                    6.014789e-10
    Vegetables, fruit, nuts                              5.163766e-10
    Oil seeds                                            9.328031e-10
                                                             ...     
WM  Membership organisation services n.e.c. (91)         1.548131e-10
    Recreational, cultural and sporting services (92)    4.536195e-10
    Other services (93)                                  8.954095e-10
    Private households with employed persons (95)        1.587403e-10
    Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [76]:
pd.DataFrame(pdf_total_s).to_csv("csv/pdf-eutrophication-freshwater-soil.csv", index=True)